<a href="https://colab.research.google.com/github/NACHAMMAI-SN/UAV-Dynamic-Programming-Path-Planning/blob/main/UAV_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly scipy matplotlib numpy -q
!pip install ipywidgets -q

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.interpolate import CubicSpline
import time
import warnings
import sys

import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

print(" Packages imported successfully!")
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")

In [ ]:

print(" Installing required packages...")

!pip install plotly ipywidgets -q

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.interpolate import CubicSpline

import time
import warnings

import plotly.graph_objects as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100


# 1. DEM GENERATION MODULE

class DEMGenerator:

    @staticmethod
    def diamond_square(shape, roughness=0.7, random_seed=None):

        if random_seed is not None:
            np.random.seed(random_seed)

        size = max(shape[0], shape[1])
        size = 2 ** int(np.ceil(np.log2(size))) + 1

        height_map = np.zeros((size, size))
        height_map[0, 0] = height_map[0, -1] = height_map[-1, 0] = height_map[-1, -1] = np.random.random()

        step = size - 1
        current_roughness = roughness

        while step > 1:
            half = step // 2

            # Diamond step
            for i in range(half, size, step):
                for j in range(half, size, step):
                    height_map[i, j] = (height_map[i-half, j-half] +
                                      height_map[i-half, j+half] +
                                      height_map[i+half, j-half] +
                                      height_map[i+half, j+half]) / 4 + np.random.uniform(-1, 1) * current_roughness

            # Square step
            for i in range(0, size, half):
                for j in range((i+half) % step, size, step):
                    total = 0
                    count = 0

                    if i >= half:
                        total += height_map[i-half, j]
                        count += 1
                    if i + half < size:
                        total += height_map[i+half, j]
                        count += 1
                    if j >= half:
                        total += height_map[i, j-half]
                        count += 1
                    if j + half < size:
                        total += height_map[i, j+half]
                        count += 1

                    height_map[i, j] = total / count + np.random.uniform(-1, 1) * current_roughness

            step //= 2
            current_roughness *= 0.5

        height_map = height_map[:shape[0], :shape[1]]
        height_map = (height_map - height_map.min()) / (height_map.max() - height_map.min()) * 100

        return height_map

    @staticmethod
    def add_obstacles(dem, obstacles_config):

        dem_with_obstacles = dem.copy()

        for obstacle in obstacles_config:
            start_row, start_col, height, width, elevation = obstacle
            end_row = min(start_row + height, dem.shape[0])
            end_col = min(start_col + width, dem.shape[1])

            dem_with_obstacles[start_row:end_row, start_col:end_col] += elevation

        return dem_with_obstacles

    @staticmethod
    def smooth_dem(dem, sigma=1.0):
        return ndimage.gaussian_filter(dem, sigma=sigma)

# 2. DYNAMIC PROGRAMMING PATH PLANNER

class DynamicProgrammingPlanner:

    def __init__(self, dem, movement_type='8-connected', elevation_weight=0.5, slope_penalty_weight=5.0):

        self.dem = dem
        self.rows, self.cols = dem.shape
        self.movement_type = movement_type
        self.elevation_weight = elevation_weight
        self.slope_penalty_weight = slope_penalty_weight

        if movement_type == '4-connected':
            self.moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # N, S, W, E
        else:  # 8-connected
            self.moves = [(-1, 0), (1, 0), (0, -1), (0, 1),      # N, S, W, E
                         (-1, -1), (-1, 1), (1, -1), (1, 1)]     # NW, NE, SW, SE

    def is_valid_position(self, pos):
        """Check if position is within DEM bounds"""
        row, col = pos
        return 0 <= row < self.rows and 0 <= col < self.cols

    def calculate_transition_cost(self, current_pos, next_pos):
        """
        Calculate cost for moving between two positions
        Cost = distance + elevation_change_penalty + slope_penalty
        """
        row1, col1 = current_pos
        row2, col2 = next_pos

        distance = np.sqrt((row2 - row1)**2 + (col2 - col1)**2)

        elevation_diff = abs(self.dem[row2, col2] - self.dem[row1, col1])
        elevation_penalty = elevation_diff * self.elevation_weight

        slope_penalty = 0
        if distance > 0:
            slope = elevation_diff / distance
            if slope > 1.0:  # Steep slope threshold
                slope_penalty = slope * self.slope_penalty_weight

        total_cost = distance + elevation_penalty + slope_penalty

        return total_cost

    def forward_dynamic_programming(self, start_pos, goal_pos, max_iterations=2000):
        start_time = time.time()

        cost = np.full((self.rows, self.cols), np.inf)
        predecessor = np.full((self.rows, self.cols, 2), -1, dtype=int)

        goal_row, goal_col = goal_pos
        cost[goal_row, goal_col] = 0

        positions = [(i, j) for i in range(self.rows) for j in range(self.cols)]
        positions.sort(key=lambda pos: np.sqrt((pos[0]-goal_row)**2 + (pos[1]-goal_col)**2))

        iteration = 0
        changed = True

        while changed and iteration < max_iterations:
            changed = False
            iteration += 1

            for current_pos in positions:
                current_row, current_col = current_pos

                if current_pos == goal_pos:
                    continue

                for move in self.moves:
                    next_pos = (current_row + move[0], current_col + move[1])

                    if self.is_valid_position(next_pos):
                        next_row, next_col = next_pos

                        # Calculate new cost via this neighbor
                        move_cost = self.calculate_transition_cost(current_pos, next_pos)
                        new_cost = cost[next_row, next_col] + move_cost

                        # Update if we found a better path
                        if new_cost < cost[current_row, current_col]:
                            cost[current_row, current_col] = new_cost
                            predecessor[current_row, current_col] = [next_row, next_col]
                            changed = True

            if iteration % 500 == 0:
                print(f"  Iteration {iteration}, still converging...")

        computation_time = time.time() - start_time
        print(f" DP completed in {iteration} iterations ({computation_time:.2f}s)")

        # Reconstruct path
        path = self._reconstruct_path(start_pos, goal_pos, predecessor)

        return path, cost

    def _reconstruct_path(self, start_pos, goal_pos, predecessor):
        path = []
        current_pos = start_pos

        max_steps = self.rows * self.cols  # Prevent infinite loops
        steps = 0

        while current_pos != goal_pos and steps < max_steps:
            path.append(current_pos)
            current_row, current_col = current_pos
            if predecessor[current_row, current_col, 0] == -1:
                print(f" No complete path found from {start_pos} to {goal_pos}")
                return None

            current_pos = (predecessor[current_row, current_col, 0],
                          predecessor[current_row, current_col, 1])
            steps += 1

        if current_pos == goal_pos:
            path.append(goal_pos)
            print(f" Path found with {len(path)} waypoints")
            return path
        else:
            print(" Path reconstruction failed")
            return None

    def calculate_path_metrics(self, path):
        """Calculate various metrics for the path"""
        if not path or len(path) < 2:
            return None

        total_distance = 0
        total_elevation_change = 0
        max_elevation = -np.inf
        min_elevation = np.inf
        max_slope = 0

        for i in range(len(path) - 1):
            row1, col1 = path[i]
            row2, col2 = path[i+1]

            # Distance
            distance = np.sqrt((row2 - row1)**2 + (col2 - col1)**2)
            total_distance += distance

            # Elevation
            elev1 = self.dem[row1, col1]
            elev2 = self.dem[row2, col2]
            elevation_change = abs(elev2 - elev1)
            total_elevation_change += elevation_change

            # Slope
            if distance > 0:
                slope = elevation_change / distance
                max_slope = max(max_slope, slope)

            max_elevation = max(max_elevation, elev1, elev2)
            min_elevation = min(min_elevation, elev1, elev2)

        metrics = {
            'waypoints': len(path),
            'total_distance': total_distance,
            'total_elevation_change': total_elevation_change,
            'max_elevation': max_elevation,
            'min_elevation': min_elevation,
            'max_slope': max_slope,
            'average_step_length': total_distance / (len(path) - 1)
        }

        return metrics

# 3. PATH SMOOTHING AND OPTIMIZATION

class PathOptimizer:
    @staticmethod
    def simple_smoothing(path, dem, smoothing_factor=0.3):

        if not path or len(path) < 3:
            return path

        smoothed_path = [path[0]]  # Keep start point

        for i in range(1, len(path) - 1):
            current = path[i]
            prev = path[i-1]
            next_p = path[i+1]

            smooth_x = (prev[0] * smoothing_factor + current[0] * (1 - 2*smoothing_factor) +
                       next_p[0] * smoothing_factor)
            smooth_y = (prev[1] * smoothing_factor + current[1] * (1 - 2*smoothing_factor) +
                       next_p[1] * smoothing_factor)

            smooth_pos = (int(round(smooth_x)), int(round(smooth_y)))

            if (0 <= smooth_pos[0] < dem.shape[0] and
                0 <= smooth_pos[1] < dem.shape[1]):
                smoothed_path.append(smooth_pos)
            else:
                smoothed_path.append(current)

        smoothed_path.append(path[-1])  # Keep end point
        return smoothed_path

    @staticmethod
    def cubic_spline_smoothing(path, dem, num_points=None):

        if not path or len(path) < 4:
            return path

        if num_points is None:
            num_points = len(path) * 2

        y_coords, x_coords = zip(*path)

        t = np.zeros(len(path))
        for i in range(1, len(path)):
            dy = y_coords[i] - y_coords[i-1]
            dx = x_coords[i] - x_coords[i-1]
            t[i] = t[i-1] + np.sqrt(dx**2 + dy**2)

        # Create splines
        t_new = np.linspace(0, t[-1], num_points)

        try:
            cs_x = CubicSpline(t, x_coords)
            cs_y = CubicSpline(t, y_coords)

            x_new = cs_x(t_new)
            y_new = cs_y(t_new)

            # Convert back to integer coordinates and ensure validity
            smoothed_path = []
            for x, y in zip(x_new, y_new):
                x_int, y_int = int(round(x)), int(round(y))
                if (0 <= y_int < dem.shape[0] and 0 <= x_int < dem.shape[1]):
                    smoothed_path.append((y_int, x_int))

            # Ensure start and end are preserved
            if smoothed_path:
                smoothed_path[0] = path[0]
                smoothed_path[-1] = path[-1]

            return smoothed_path

        except Exception as e:
            print(f"Spline smoothing failed: {e}")
            return path

    @staticmethod
    def remove_redundant_points(path, tolerance=1.0):

        if len(path) < 3:
            return path

        simplified = [path[0]]

        for i in range(1, len(path) - 1):
            prev = simplified[-1]
            current = path[i]
            next_p = path[i+1]

            dist_to_line = PathOptimizer._point_to_line_distance(prev, next_p, current)

            if dist_to_line > tolerance:
                simplified.append(current)

        simplified.append(path[-1])
        return simplified

    @staticmethod
    def _point_to_line_distance(line_start, line_end, point):
        """Calculate distance from point to line segment"""
        x0, y0 = point[1], point[0]
        x1, y1 = line_start[1], line_start[0]
        x2, y2 = line_end[1], line_end[0]

        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2-y1)**2 + (x2-x1)**2)

        if denominator == 0:
            return np.sqrt((x0-x1)**2 + (y0-y1)**2)

        return numerator / denominator

# 4. INTERACTIVE VISUALIZATION MODULE

class InteractiveVisualization:

    @staticmethod
    def create_interactive_dem_plot(dem, path=None, smoothed_path=None, start_pos=None, goal_pos=None, title="UAV Path Planning"):
        x = np.arange(dem.shape[1])
        y = np.arange(dem.shape[0])
        X, Y = np.meshgrid(x, y)

        fig = go.Figure()

        # Add DEM surface
        fig.add_trace(go.Surface(
            x=X, y=Y, z=dem,
            colorscale='Viridis',
            opacity=0.8,
            name='Terrain',
            showscale=True
        ))

        # Add original path if provided
        if path:
            path_y, path_x = zip(*path)
            path_z = [dem[y, x] + 2 for y, x in path]  # Offset for visibility

            fig.add_trace(go.Scatter3d(
                x=path_x, y=path_y, z=path_z,
                mode='lines+markers',
                line=dict(color='red', width=6),
                marker=dict(size=3, color='red'),
                name='Original Path'
            ))

        if smoothed_path:
            smooth_y, smooth_x = zip(*smoothed_path)
            smooth_z = [dem[y, x] + 3 for y, x in smoothed_path]  # Different offset

            fig.add_trace(go.Scatter3d(
                x=smooth_x, y=smooth_y, z=smooth_z,
                mode='lines+markers',
                line=dict(color='magenta', width=8),
                marker=dict(size=4, color='magenta'),
                name='Smoothed Path'
            ))

        if start_pos:
            start_z = dem[start_pos[0], start_pos[1]] + 5
            fig.add_trace(go.Scatter3d(
                x=[start_pos[1]], y=[start_pos[0]], z=[start_z],
                mode='markers',
                marker=dict(size=10, color='green', symbol='diamond'),
                name='Start'
            ))

        if goal_pos:
            goal_z = dem[goal_pos[0], goal_pos[1]] + 5
            fig.add_trace(go.Scatter3d(
                x=[goal_pos[1]], y=[goal_pos[0]], z=[goal_z],
                mode='markers',
                marker=dict(size=10, color='blue', symbol='circle'),  # Changed from 'star' to 'circle'
                name='Goal'
            ))

        fig.update_layout(
            title=dict(text=title, x=0.5, xanchor='center'),
            scene=dict(
                xaxis_title='X Coordinate',
                yaxis_title='Y Coordinate',
                zaxis_title='Elevation',
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            width=800,
            height=600
        )

        return fig

    @staticmethod
    def plot_comprehensive_results(dem, original_path, smoothed_path, start_pos, goal_pos, cost_map):
        """Create comprehensive visualization of all results"""
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))

        # 1. Original DEM
        ax1 = axes[0, 0]
        im1 = ax1.imshow(dem, cmap='terrain', aspect='auto')
        ax1.set_title('Digital Elevation Model (DEM)', fontsize=14, fontweight='bold')
        ax1.set_xlabel('X Coordinate')
        ax1.set_ylabel('Y Coordinate')
        plt.colorbar(im1, ax=ax1, label='Elevation')

        # 2. Cost Map
        ax2 = axes[0, 1]
        im2 = ax2.imshow(np.where(cost_map == np.inf, np.nan, cost_map),
                        cmap='viridis', aspect='auto')
        ax2.set_title('Dynamic Programming Cost Map', fontsize=14, fontweight='bold')
        ax2.set_xlabel('X Coordinate')
        ax2.set_ylabel('Y Coordinate')
        plt.colorbar(im2, ax=ax2, label='Cost to Goal')

        # 3. Original Path
        ax3 = axes[0, 2]
        ax3.imshow(dem, cmap='terrain', aspect='auto')
        if original_path:
            path_y, path_x = zip(*original_path)
            ax3.plot(path_x, path_y, 'r-', linewidth=3, label='Original Path', alpha=0.8)
            ax3.plot(path_x, path_y, 'ro', markersize=2, alpha=0.6)
        ax3.plot(start_pos[1], start_pos[0], 'g^', markersize=15, label='Start', markerfacecolor='green')
        ax3.plot(goal_pos[1], goal_pos[0], 'b*', markersize=15, label='Goal', markerfacecolor='blue')
        ax3.set_title('Original DP Path', fontsize=14, fontweight='bold')
        ax3.legend()

        # 4. Smoothed Path
        ax4 = axes[1, 0]
        ax4.imshow(dem, cmap='terrain', aspect='auto')
        if smoothed_path:
            smooth_y, smooth_x = zip(*smoothed_path)
            ax4.plot(smooth_x, smooth_y, 'm-', linewidth=3, label='Smoothed Path')
            ax4.plot(smooth_x, smooth_y, 'mo', markersize=2, alpha=0.6)
        ax4.plot(start_pos[1], start_pos[0], 'g^', markersize=15, label='Start')
        ax4.plot(goal_pos[1], goal_pos[0], 'b*', markersize=15, label='Goal')
        ax4.set_title('Optimized Smooth Path', fontsize=14, fontweight='bold')
        ax4.legend()

        # 5. Path Comparison
        ax5 = axes[1, 1]
        ax5.imshow(dem, cmap='terrain', aspect='auto')
        if original_path:
            ax5.plot(path_x, path_y, 'r-', linewidth=2, label='Original Path', alpha=0.7)
        if smoothed_path:
            ax5.plot(smooth_x, smooth_y, 'm-', linewidth=3, label='Smoothed Path')
        ax5.plot(start_pos[1], start_pos[0], 'g^', markersize=15, label='Start')
        ax5.plot(goal_pos[1], goal_pos[0], 'b*', markersize=15, label='Goal')
        ax5.set_title('Path Comparison', fontsize=14, fontweight='bold')
        ax5.legend()

        # 6. 3D Visualization
        ax6 = axes[1, 2]
        # Create simple 3D plot
        from mpl_toolkits.mplot3d import Axes3D
        x = np.arange(dem.shape[1])
        y = np.arange(dem.shape[0])
        X, Y = np.meshgrid(x, y)

        # Plot terrain surface
        ax6 = fig.add_subplot(236, projection='3d')
        surf = ax6.plot_surface(X, Y, dem, cmap='terrain', alpha=0.7,
                               linewidth=0, antialiased=True)

        # Plot smoothed path in 3D
        if smoothed_path:
            path_x_3d, path_y_3d, path_z_3d = [], [], []
            for pos in smoothed_path:
                path_x_3d.append(pos[1])
                path_y_3d.append(pos[0])
                path_z_3d.append(dem[pos[0], pos[1]] + 2)  # Offset for visibility

            ax6.plot(path_x_3d, path_y_3d, path_z_3d, 'm-', linewidth=4, label='UAV Path')
            ax6.plot([path_x_3d[0]], [path_y_3d[0]], [path_z_3d[0]],
                    'g^', markersize=10, label='Start')
            ax6.plot([path_x_3d[-1]], [path_y_3d[-1]], [path_z_3d[-1]],
                    'b*', markersize=15, label='Goal')

        ax6.set_title('3D Path Visualization', fontsize=14, fontweight='bold')
        ax6.set_xlabel('X Coordinate')
        ax6.set_ylabel('Y Coordinate')
        ax6.set_zlabel('Elevation')
        ax6.legend()

        plt.tight_layout()
        plt.show()

# 5. INTERACTIVE MAIN SYSTEM

def run_interactive_uav_system(
    dem_size=80,
    terrain_roughness=0.7,
    smoothing_sigma=1.2,
    start_x=8, start_y=8,
    goal_x=70, goal_y=70,
    elevation_weight=0.6,
    slope_penalty=5.0,
    movement_type='8-connected',
    simple_smoothing_factor=0.2,
    spline_points=150,
    simplification_tolerance=1.2,
    add_mountain_1=True, mountain_1_params=(30, 20, 15, 12, 45),
    add_mountain_2=True, mountain_2_params=(10, 50, 12, 10, 35),
    add_mountain_3=True, mountain_3_params=(50, 60, 10, 15, 40),
    add_hill=True, hill_params=(60, 15, 8, 12, 30)
):

    print("=" * 70)
    print(" INTERACTIVE UAV PATH PLANNING SYSTEM")
    print("=" * 70)

    # Step 1: Generate DEM with interactive parameters
    print("\n1.  GENERATING DIGITAL ELEVATION MODEL...")
    dem_gen = DEMGenerator()

    # Generate base DEM
    dem = dem_gen.diamond_square((dem_size, dem_size), roughness=terrain_roughness, random_seed=42)

    # Add obstacles based on interactive parameters
    obstacles = []
    if add_mountain_1:
        obstacles.append(mountain_1_params)
    if add_mountain_2:
        obstacles.append(mountain_2_params)
    if add_mountain_3:
        obstacles.append(mountain_3_params)
    if add_hill:
        obstacles.append(hill_params)

    dem = dem_gen.add_obstacles(dem, obstacles)
    dem = dem_gen.smooth_dem(dem, sigma=smoothing_sigma)

    print(f"   DEM Size: {dem.shape}")
    print(f"   Elevation Range: {dem.min():.1f} - {dem.max():.1f}")
    print(f"   Obstacles Added: {len(obstacles)}")

    # Step 2: Initialize Path Planner
    print("\n2.  INITIALIZING DYNAMIC PROGRAMMING PLANNER...")
    planner = DynamicProgrammingPlanner(
        dem=dem,
        movement_type=movement_type,
        elevation_weight=elevation_weight,
        slope_penalty_weight=slope_penalty
    )

    # Step 3: Define Start and Goal Positions
    start_pos = (start_y, start_x)  # Note: (row, col) format
    goal_pos = (goal_y, goal_x)

    print(f"   Start Position: ({start_x}, {start_y})")
    print(f"   Goal Position: ({goal_x}, {goal_y})")
    print(f"   Movement Type: {movement_type}")
    print(f"   Elevation Weight: {elevation_weight}")
    print(f"   Slope Penalty: {slope_penalty}")

    # Step 4: Compute Optimal Path using Dynamic Programming
    print("\n3.  COMPUTING OPTIMAL PATH...")
    path, cost_map = planner.forward_dynamic_programming(start_pos, goal_pos)
    print(cost_map)
    print(len(cost_map))

    if path is None:
        print(" No valid path found! Try different start/goal positions or reduce obstacles.")
        return None, None, None, None

    # Step 5: Optimize and Smooth the Path
    print("\n4. OPTIMIZING AND SMOOTHING PATH...")
    optimizer = PathOptimizer()

    # Apply multiple smoothing techniques
    smoothed_path = optimizer.simple_smoothing(path, dem, smoothing_factor=simple_smoothing_factor)
    smoothed_path = optimizer.cubic_spline_smoothing(smoothed_path, dem, num_points=spline_points)
    smoothed_path = optimizer.remove_redundant_points(smoothed_path, tolerance=simplification_tolerance)

    print(f"   Original waypoints: {len(path)}")
    print(f"   Smoothed waypoints: {len(smoothed_path)}")
    print(f"   Smoothing Factor: {simple_smoothing_factor}")
    print(f"   Spline Points: {spline_points}")
    print(f"   Simplification Tolerance: {simplification_tolerance}")

    # Step 6: Calculate Path Metrics
    print("\n5. CALCULATING PATH METRICS...")
    original_metrics = planner.calculate_path_metrics(path)
    smoothed_metrics = planner.calculate_path_metrics(smoothed_path)

    if original_metrics and smoothed_metrics:
        print("\n--- ORIGINAL PATH METRICS ---")
        print(f"   Waypoints: {original_metrics['waypoints']}")
        print(f"   Total Distance: {original_metrics['total_distance']:.2f}")
        print(f"   Total Elevation Change: {original_metrics['total_elevation_change']:.2f}")
        print(f"   Max Slope: {original_metrics['max_slope']:.2f}")

        print("\n--- SMOOTHED PATH METRICS ---")
        print(f"   Waypoints: {smoothed_metrics['waypoints']}")
        print(f"   Total Distance: {smoothed_metrics['total_distance']:.2f}")
        print(f"   Total Elevation Change: {smoothed_metrics['total_elevation_change']:.2f}")
        print(f"   Max Slope: {smoothed_metrics['max_slope']:.2f}")

        # Calculate improvements
        distance_improvement = ((original_metrics['total_distance'] - smoothed_metrics['total_distance']) /
                              original_metrics['total_distance'] * 100)
        waypoint_reduction = ((original_metrics['waypoints'] - smoothed_metrics['waypoints']) /
                            original_metrics['waypoints'] * 100)

        print(f"\n    IMPROVEMENT SUMMARY:")
        print(f"   Distance Improvement: {distance_improvement:+.1f}%")
        print(f"   Waypoint Reduction: {waypoint_reduction:+.1f}%")

    # Step 7: Interactive Visualizations
    print("\n6.  GENERATING INTERACTIVE VISUALIZATIONS...")
    viz = InteractiveVisualization()

    # Create comprehensive static plots
    viz.plot_comprehensive_results(dem, path, smoothed_path, start_pos, goal_pos, cost_map)

    # Create interactive 3D plot
    interactive_fig = viz.create_interactive_dem_plot(
        dem, path, smoothed_path, start_pos, goal_pos,
        title=f"UAV Path Planning - {movement_type} Movement"
    )
    interactive_fig.show()

    # Step 8: Export Results
    print("\n7.  EXPORTING RESULTS...")

    def save_waypoints(path, filename):
        with open(filename, 'w') as f:
            f.write("Waypoint,X,Y,Elevation\n")
            for i, (row, col) in enumerate(path):
                f.write(f"{i},{col},{row},{dem[row, col]:.2f}\n")
        print(f"    Waypoints saved to {filename}")

    save_waypoints(smoothed_path, "uav_waypoints.csv")

    # Save DEM data
    np.savetxt("dem_data.csv", dem, delimiter=",", fmt="%.2f")
    print("    DEM data saved to dem_data.csv")

    print("\n" + "=" * 70)
    print(" INTERACTIVE UAV PATH PLANNING COMPLETED SUCCESSFULLY!")
    print("=" * 70)

    return dem, path, smoothed_path, smoothed_metrics

# 6. CREATE INTERACTIVE WIDGETS

def create_interactive_interface():
    """Create interactive widgets for the UAV system"""

    print(" CREATING INTERACTIVE UAV PATH PLANNING INTERFACE...")
    print("Adjust the parameters below and click 'Run UAV Planning' to see results!")

    # Create interactive widgets
    style = {'description_width': 'initial'}

    interact_manual(
        run_interactive_uav_system,
        dem_size=widgets.IntSlider(value=80, min=50, max=150, step=10, description='DEM Size:', style=style),
        terrain_roughness=widgets.FloatSlider(value=0.7, min=0.1, max=1.5, step=0.1, description='Terrain Roughness:', style=style),
        smoothing_sigma=widgets.FloatSlider(value=1.2, min=0.5, max=3.0, step=0.1, description='DEM Smoothing:', style=style),

        start_x=widgets.IntSlider(value=8, min=5, max=75, step=5, description='Start X:', style=style),
        start_y=widgets.IntSlider(value=8, min=5, max=75, step=5, description='Start Y:', style=style),
        goal_x=widgets.IntSlider(value=70, min=10, max=75, step=5, description='Goal X:', style=style),
        goal_y=widgets.IntSlider(value=70, min=10, max=75, step=5, description='Goal Y:', style=style),

        elevation_weight=widgets.FloatSlider(value=0.6, min=0.1, max=2.0, step=0.1, description='Elevation Weight:', style=style),
        slope_penalty=widgets.FloatSlider(value=5.0, min=0.0, max=20.0, step=1.0, description='Slope Penalty:', style=style),
        movement_type=widgets.RadioButtons(options=['4-connected', '8-connected'], value='8-connected', description='Movement Type:', style=style),

        simple_smoothing_factor=widgets.FloatSlider(value=0.2, min=0.0, max=0.5, step=0.05, description='Smoothing Factor:', style=style),
        spline_points=widgets.IntSlider(value=150, min=50, max=300, step=25, description='Spline Points:', style=style),
        simplification_tolerance=widgets.FloatSlider(value=1.2, min=0.5, max=3.0, step=0.1, description='Simplification Tolerance:', style=style),

        add_mountain_1=widgets.Checkbox(value=True, description='Add Mountain 1', style=style),
        mountain_1_params=widgets.fixed((30, 20, 15, 12, 45)),
        add_mountain_2=widgets.Checkbox(value=True, description='Add Mountain 2', style=style),
        mountain_2_params=widgets.fixed((10, 50, 12, 10, 35)),
        add_mountain_3=widgets.Checkbox(value=True, description='Add Mountain 3', style=style),
        mountain_3_params=widgets.fixed((50, 60, 10, 15, 40)),
        add_hill=widgets.Checkbox(value=True, description='Add Hill', style=style),
        hill_params=widgets.fixed((60, 15, 8, 12, 30))
    )

# 7. EXECUTE THE INTERACTIVE SYSTEM
print(" UAV PATH PLANNING WITH DYNAMIC PROGRAMMING")
create_interactive_interface()

print("\n RUN WITH DEFAULT PARAMETERS:")
print("Click to run")
